Here are the dependencies, the last two are new

`pip install numpy==1.25 fairlearn==0.9.0 plotly==5.24.1 nbformat==5.10.4 aif360['inFairness']==0.6.1 ipykernel==6.29.5 BlackBoxAuditing==0.1.54 cvxpy==1.6.0 `



!!! Attention sur Colab!!!, après avoir executé la cellule ci-dessus, il faudra redémarrer la session (onglet "Execution") afin de charger l'environnement installé

# TD 3: Mitigation des biais avec des méthodes de pré-processing et de post-processing



## 1.Manipulate the dataset

In [1]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score


MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
)

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [2]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [3]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948])

In [4]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

Conversion en dataframe

In [5]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df["WEIGHT"] = data[1]["instance_weights"]
    return df


df = get_df(MEPSDataset19_data)

Nous réalisons maintenant l'opération inverse (qui sera indispensable pour le projet). Créer un objet de la classe StandardDataset de AIF360 à partir du dataframe. 

Pour le projet cela vous permettre d'utiliser les méthode déjà implémentées dans AIF360 sur votre jeu de données.

Ici cela n'a aucun intéret car le dataframe vien d'un StandardDataset, nous vous fournissons le code. Mais cela vaut le coup de le lire attentivement et de poser des questions si besoin.




In [6]:
import os
from aif360.datasets import StandardDataset
import pandas as pd

# Get categorical column from one hot encoding (specitic to MEPSdataset)
# Here we create a dictionnary that links each categorical column name
# to the list of corresponding one hot encoded columns
categorical_columns_dic = {}
for col in df.columns:
    col_split = col.split("=")
    if len(col_split) > 1:
        cat_col = col_split[0]
        if not (cat_col in categorical_columns_dic.keys()):
            categorical_columns_dic[cat_col] = []
        categorical_columns_dic[cat_col].append(col)
categorical_features = categorical_columns_dic.keys()

In [7]:
# Now we recreate the categorical column value from the one hot encoded
print(df.shape)


def categorical_transform(df, onehotencoded, cat_col):
    if len(onehotencoded) > 1:
        return df[onehotencoded].apply(
            lambda x: onehotencoded[np.argmax(x)][len(cat_col) + 1 :], axis=1
        )
    else:
        return df[onehotencoded]


# Reverse the categorical one hot encoded
for cat_col, onehotencoded in categorical_columns_dic.items():
    df[cat_col] = categorical_transform(df, onehotencoded, cat_col)
    df.drop(columns=onehotencoded, inplace=True)

df.shape

(15830, 140)


(15830, 43)

In [8]:
MyDataset = StandardDataset(
    df=df,
    label_name="UTILIZATION",
    favorable_classes=[1],
    protected_attribute_names=["RACE"],
    privileged_classes=[[1]],
    instance_weights_name="WEIGHT",
    categorical_features=categorical_features,
    features_to_keep=[],
    features_to_drop=[],
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

In [9]:
# We check the dataset has the same metrics :D
# Attention étonnanement le positive label 'favorable_classes' est par défaut 1 (cela est un peu bizarre pour ce dataset)
print(
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).base_rate(),
)
print(
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).base_rate(),
)

0.49826823461176517 0.21507139363038463
0.49826823461176517 0.21507139363038463


In [10]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate

dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, prot_attr=df.RACE, pos_label=1, sample_weight=df.WEIGHT
)
br = base_rate(y_true=df.UTILIZATION, pos_label=1, sample_weight=df.WEIGHT)
dir, br

(0.4982682346117653, 0.21507139363038463)

### Question1: Create a function that print the fairness metrics of a dataset 

In [11]:
from aif360.sklearn.metrics import *


def get_group_metrics(
    y_true,
    y_pred=None,
    prot_attr=None,
    priv_group=1,
    pos_label=1,
    sample_weight=None,
):
    group_metrics = {}
    group_metrics["base_rate"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
    return group_metrics

In [12]:
group_metrics = get_group_metrics(
        y_true= df.UTILIZATION,
        y_pred= None,
        prot_attr= df.RACE,
        pos_label= 1,
        sample_weight= df.WEIGHT,
)
group_metrics

{'base_rate': 0.21507139363038463,
 'statistical_parity_difference': -0.13507447726478136,
 'disparate_impact_ratio': 0.4982682346117653}

## 2. Appliquer les méthodes de pré-processing disponibles dans AIF360

In [13]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

('RACE', [{'RACE': 0.0}], [{'RACE': 1.0}])

### 2.1 Quesiton: Apprendre une regression logistique qui prédit l'UTILIZATION

Attention nous avons enlever le preprocessing sur le dataframe, il faut cette fois utiliser l'API d'AIF360
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html

pour retrouver les features (X), les labels (y) et les poids de chaque instance du dataset

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

X_train = dataset_orig_panel19_train.features
y_train = dataset_orig_panel19_train.labels[:,0]
X_val = dataset_orig_panel19_val.features
y_val = dataset_orig_panel19_val.labels[:,0]


model = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model = model.fit(
    X_train,
    y_train,
    **{"logisticregression__sample_weight": dataset_orig_panel19_train.instance_weights}
)

preds = model.predict(X_val)

model.score(X_val, y_val, sample_weight=dataset_orig_panel19_val.instance_weights)

0.8469502635753938

In [15]:
accuracy_score(y_val, preds, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8469502635753938, 0.705306798187123)

### 2.2 Question: Calcul des métriques de fairness

Calculer les métriques du dataset de validation seul.

Calculer les métriques basées sur les prédictions et la vérité du dataset de validation.

En comparaison calculer les métriques basées sur des prédictions aléatoires et la vérité du dataset de validation.

In [16]:
# Metrics on validation dataset
get_group_metrics(
        y_true=y_val,
        y_pred=None,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.12134966576028675,
 'disparate_impact_ratio': 0.5260990512554462}

In [17]:
# Metrics based on predictions and truth on validation dataset
get_group_metrics(
        y_true=y_val,
        y_pred=preds,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.11419180291702838,
 'disparate_impact_ratio': 0.37940849083495765,
 'equal_opportunity_difference': -0.1712968207977449,
 'average_odds_difference': -0.10771665651734363,
 'conditional_demographic_disparity': -0.04463003128116544,
 'smoothed_edf': 0.969141552940342,
 'df_bias_amplification': 0.3268758988759921}

In [18]:
# Metrics based on random predictions and truth on validation dataset

pred_random = np.array([int(r*2) for r in np.random.random(size=len(y_val)).tolist()])
pred_random.max(), pred_random.min()
get_group_metrics(
        y_true=y_val,
        y_pred=pred_random,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.0035456211554041883,
 'disparate_impact_ratio': 0.9928829221410262,
 'equal_opportunity_difference': -0.09712682476440238,
 'average_odds_difference': -0.0395212623132353,
 'conditional_demographic_disparity': -0.0006591341968137991,
 'smoothed_edf': 0.007142524585142529,
 'df_bias_amplification': -0.6351231294792073}

### 2.2 Repondération
#### 2.2.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire de repondération et les appliquer sur le dataset d'apprentissage

In [19]:
from aif360.algorithms.preprocessing import *

RW = Reweighing(
    unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
)

In [20]:
RW.fit(dataset_orig_panel19_train)
dataset_transf_train = RW.transform(dataset_orig_panel19_train)
dataset_transf_val = RW.transform(dataset_orig_panel19_val)

#### 2.2.2. Question: Apprendre une regression logistique sur les données pondérées et calculer les métriques de fairness sur l'échantillon de validation

Comme vu en cours le Reweighting ne modifie que la pondération du dataset, les features et label restent inchangés.

In [21]:
model_rw = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_rw = model_rw.fit(
    X_train,
    y_train,
    **{"logisticregression__sample_weight": dataset_transf_train.instance_weights}
)

preds_rw = model_rw.predict(X_val)

model_rw.score(X_val, y_val), model_rw.score(X_val, y_val, sample_weight=dataset_transf_val.instance_weights)

(0.8677616340282165, 0.8468268453442269)

In [22]:
accuracy_score(y_val, preds_rw, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_rw, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.84889536077611, 0.7044360411962385)

In [23]:
# Metrics on prediction with RW on validation dataset
get_group_metrics(
    y_true=y_val,
    y_pred=preds_rw,
    prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=dataset_transf_val.instance_weights,
)

{'base_rate': 0.2095619112874149,
 'statistical_parity_difference': 0.0006922227730098707,
 'disparate_impact_ratio': 1.0051842240054536,
 'equal_opportunity_difference': -0.025632212931009424,
 'average_odds_difference': -0.012963255560900694,
 'conditional_demographic_disparity': 0.00028098957787494683,
 'smoothed_edf': 0.005170883017437067,
 'df_bias_amplification': -0.06936987668262229}

### 2.3. Disparate Impact Remover
#### 2.3.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire une approache de disparate impact remover et les appliquer. 


In [24]:
DIR = DisparateImpactRemover(repair_level=1., sensitive_attribute='RACE')
train_dir = DIR.fit_transform(dataset_orig_panel19_train)
val_dir = DIR.fit_transform(dataset_orig_panel19_val)

In [25]:
dataset_orig_panel19_train

               instance weights features                                    \
                                         protected attribute                 
                                     AGE                RACE  PCS42  MCS42   
instance names                                                               
13074               4854.422747     13.0                 0.0  -1.00  -1.00   
598                 6491.605615     56.0                 0.0  32.00  63.40   
12539               5746.605598     47.0                 0.0  55.09  56.74   
2337               11967.358980     85.0                 1.0  35.24  44.77   
1487               17276.935353     25.0                 1.0  56.15  57.16   
...                         ...      ...                 ...    ...    ...   
605                 3825.982645      2.0                 1.0  -1.00  -1.00   
5317                1031.365179     29.0                 0.0  -1.00  -1.00   
15974               7555.375410     41.0                 0.0  51

In [26]:
train_dir

               instance weights features                                    \
                                         protected attribute                 
                                     AGE                RACE  PCS42  MCS42   
instance names                                                               
13074               4854.422747     13.0                 0.0  -1.00  -1.00   
598                 6491.605615     56.0                 0.0  29.10  63.23   
12539               5746.605598     47.0                 0.0  55.09  56.74   
2337               11967.358980     85.0                 1.0  35.24  44.42   
1487               17276.935353     25.0                 1.0  56.01  57.16   
...                         ...      ...                 ...    ...    ...   
605                 3825.982645      2.0                 1.0  -1.00  -1.00   
5317                1031.365179     29.0                 0.0  -1.00  -1.00   
15974               7555.375410     41.0                 0.0  50

#### 2.3.2. Question: Apprendre une regression logistique sur les données transformées en retirant l'attribut sensible et calculer les métriques de fairness sur l'échantillon de validation

In [27]:
protected="RACE"
index = dataset_orig_panel19_train.feature_names.index(protected)
model_dir = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_dir = model_dir.fit(
    np.delete(train_dir.features, index, axis=1),
    train_dir.labels[:,0],
    **{"logisticregression__sample_weight": train_dir.instance_weights}
)

preds_dir = model_dir.predict(np.delete(val_dir.features, index, axis=1))

model_dir.score(np.delete(val_dir.features, index, axis=1), val_dir.labels[:,0], sample_weight=val_dir.instance_weights)

0.8514288533074994

In [28]:
accuracy_score(y_val, preds_dir, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_dir, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8514288533074994, 0.7129236915393814)

In [29]:
get_group_metrics(
    y_true=val_dir.labels[:,0],
    y_pred=preds_dir,
    prot_attr=val_dir.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=val_dir.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.0931466144269939,
 'disparate_impact_ratio': 0.4719865662720144,
 'equal_opportunity_difference': -0.11607039635390687,
 'average_odds_difference': -0.07166894763436302,
 'conditional_demographic_disparity': -0.03620763647608569,
 'smoothed_edf': 0.7508045374105108,
 'df_bias_amplification': 0.10853888334616091}

### 2.4. Question: Apprentissage de représentation latente fair

Apprendre le pre-processing et evaluer son impact avec les métriques

In [30]:
from aif360.algorithms.preprocessing.lfr import LFR
TR = LFR(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    k=5,
    Ax=0.01,
    Ay=1.0,
    Az=50.0,
    print_interval=250,
    verbose=1,
    seed=None,
)
TR = TR.fit(dataset_orig_panel19_train, maxiter=5000, maxfun=5000)

step: 0, loss: 1.4659595773313123, L_x: 73.94598829174177,  L_y: 0.49051756409957825,  L_z: 0.004719642606286328
step: 250, loss: 1.46595971088572, L_x: 73.94598833018088,  L_y: 0.49051756006723446,  L_z: 0.00471964535033354
step: 500, loss: 1.4659595696651782, L_x: 73.94598828277832,  L_y: 0.49051756446983963,  L_z: 0.004719642447351108
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          695     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.46596D+00    |proj g|=  2.04111D-01
step: 750, loss: 1.272585756019214, L_x: 73.89492042874119,  L_y: 0.46301402602024677,  L_z: 0.001412450514231106
step: 1000, loss: 1.2725855361078842, L_x: 73.89492042960904,  L_y: 0.46301402461390384,  L_z: 0.0014124461439578018
step: 1250, loss: 1.2725853566771945, L_x: 73.8949204625469,  L_y: 0.4630140320109748,  L_z: 0.0014124424008150117

At iterate    1    f=  1.27259D+00    |proj g|=  1.71041D-01
step: 1500, loss: 1.737

In [31]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_panel19_train)
dataset_transf_val = TR.transform(dataset_orig_panel19_val)

In [32]:
model_lfr = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_lfr = model_lfr.fit(
    dataset_transf_train.features,
    y_train,
    **{"logisticregression__sample_weight": dataset_transf_train.instance_weights}
)

preds_lfr = model_lfr.predict(dataset_transf_val.features)

model_lfr.score(dataset_transf_val.features, y_val, sample_weight=dataset_transf_val.instance_weights)

0.7922054620722778

In [33]:
accuracy_score(y_val, preds_lfr, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_lfr, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.7922054620722778, 0.4995661116350048)

In [34]:
get_group_metrics(
            y_true=y_val,
            y_pred=preds_lfr,
            prot_attr=dataset_transf_val.protected_attributes[:, sens_ind],
            pos_label=1,
            sample_weight=dataset_transf_val.instance_weights,
        )

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.0011533968392891188,
 'disparate_impact_ratio': 0.0,
 'equal_opportunity_difference': 0.0,
 'average_odds_difference': -0.0007752004925231093,
 'conditional_demographic_disparity': -0.0779575236497689,
 'smoothed_edf': 9.900830448204257,
 'df_bias_amplification': 9.258564794139907}

## 3 Post processing

### 3.1 Question: Use the post-processing Reject Option Classification

In [35]:
from aif360.algorithms.postprocessing.reject_option_classification import (
    RejectOptionClassification,
)


#### 3.1.1 Reuse the first Logistic Regression learn to find the best threshold that maximises its balanced accuracy on the validation dataset

In [36]:
# Find the best classification threshold of the validation dataset
df_val_pred = dataset_orig_panel19_val.copy(deepcopy=True)
df_val_pred.scores = model.predict_proba(dataset_orig_panel19_val.features)[:,1].reshape(-1,1)
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = df_val_pred.scores > class_thresh
    df_val_pred.labels[fav_inds] = df_val_pred.favorable_label
    df_val_pred.labels[~fav_inds] = df_val_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_panel19_val,
                                             df_val_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

In [37]:
f" best indice {best_ind}, corresponding balanced accuracy {ba_arr[best_ind]}, and threshold {best_class_thresh}"

' best indice 23, corresponding balanced accuracy 0.7623408122393942, and threshold 0.23767676767676768'

#### 3.1.2 Use the RejectOptionClassification  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [38]:
metric_name = "Statistical parity difference"
metric_ub = 0.05
metric_lb = -0.05

ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)

ROC = ROC.fit(dataset_orig_panel19_val, df_val_pred)

In [39]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.2080
Optimal ROC margin = 0.0764


In [40]:
df_roc_val_pred = ROC.predict(df_val_pred)
get_group_metrics(
    y_true=y_val,
    y_pred=df_roc_val_pred.labels[:,0],
    prot_attr=df_val_pred.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.04249645116073564,
 'disparate_impact_ratio': 0.8618525286313132,
 'equal_opportunity_difference': 0.05410668027317256,
 'average_odds_difference': 0.03430762918665692,
 'conditional_demographic_disparity': -0.009582586906338744,
 'smoothed_edf': 0.14867107665762447,
 'df_bias_amplification': -0.4935945774067254}

In [41]:
accuracy_score(y_val, df_roc_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_roc_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.7940235015673595, 0.7607647568837854)

#### 3.1.3 Do the same while starting from the Logistic Regression learned on the Reweighted dataset

In [42]:
# Find the best classification threshold of the validation dataset
df_val_pred_rw = dataset_orig_panel19_val.copy(deepcopy=True)
df_val_pred_rw.scores = model_rw.predict_proba(dataset_orig_panel19_val.features)[:,1].reshape(-1,1)
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = df_val_pred_rw.scores > class_thresh
    df_val_pred_rw.labels[fav_inds] = df_val_pred_rw.favorable_label
    df_val_pred_rw.labels[~fav_inds] = df_val_pred_rw.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_panel19_val,
                                             df_val_pred_rw, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

In [43]:
f" best indice {best_ind}, corresponding balanced accuracy {ba_arr[best_ind]}, and threshold {best_class_thresh}"

' best indice 20, corresponding balanced accuracy 0.7634407716783915, and threshold 0.207979797979798'

In [44]:
ROC_rw = ROC.fit(dataset_orig_panel19_val, df_val_pred_rw)

In [45]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC_rw.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_rw.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.2179
Optimal ROC margin = 0.0044


In [46]:
df_roc_val_pred_rw = ROC.predict(df_val_pred_rw)
get_group_metrics(
    y_true=y_val,
    y_pred=df_roc_val_pred_rw.labels[:,0],
    prot_attr=df_val_pred_rw.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred_rw.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.04490734784958361,
 'disparate_impact_ratio': 0.8561548632842189,
 'equal_opportunity_difference': 0.0589910626589043,
 'average_odds_difference': 0.03476425217100709,
 'conditional_demographic_disparity': -0.010053205876755662,
 'smoothed_edf': 0.1553039773326257,
 'df_bias_amplification': -0.4869616767317242}

In [47]:
accuracy_score(y_val, df_roc_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_roc_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.7919141161157263, 0.7607666558386222)

#### 3.2 Use the Calibrated Equalised Odds  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [48]:
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from tqdm import tqdm

# Learn parameters to equalize odds and apply to create a new dataset
# cost constraint of fnr will optimize generalized false negative rates, that of
# fpr will optimize generalized false positive rates, and weighted will optimize
# a weighted combination of both
cost_constraint = "fnr" # "fnr", "fpr", "weighted"
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=42)
cpp = cpp.fit(dataset_orig_panel19_val, df_val_pred)

In [49]:
df_ceqodds_val_pred = cpp.predict(df_val_pred)
get_group_metrics(
    y_true=y_val,
    y_pred=df_ceqodds_val_pred.labels[:,0],
    prot_attr=df_val_pred.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.0008382875124766653,
 'disparate_impact_ratio': 0.9881348544833901,
 'equal_opportunity_difference': 0.14192622345508196,
 'average_odds_difference': 0.07117364819110404,
 'conditional_demographic_disparity': -0.0005959688147473498,
 'smoothed_edf': 0.011935979500472982,
 'df_bias_amplification': -0.6303296745638769}

In [50]:
accuracy_score(y_val, df_ceqodds_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_ceqodds_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8188781744393867, 0.602265005680144)

In [51]:
cpp_rw = cpp.fit(dataset_orig_panel19_val, df_val_pred_rw)

In [52]:
df_ceqodds_val_pred_rw = cpp_rw.predict(df_val_pred_rw)
get_group_metrics(
    y_true=y_val,
    y_pred=df_ceqodds_val_pred_rw.labels[:,0],
    prot_attr=df_val_pred_rw.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred_rw.instance_weights,
)

{'base_rate': 0.20710648338464316,
 'statistical_parity_difference': -0.054124769845432505,
 'disparate_impact_ratio': 0.6518716969592314,
 'equal_opportunity_difference': -0.025632212931009035,
 'average_odds_difference': -0.012963255560900666,
 'conditional_demographic_disparity': -0.02172567428554692,
 'smoothed_edf': 0.4279073794370918,
 'df_bias_amplification': -0.2143582746272581}

In [53]:
accuracy_score(y_val, df_ceqodds_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_ceqodds_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.84889536077611, 0.7044360411962385)